In [25]:
import numpy as np
import pandas as pd
import pickle
import keras
from keras.layers import Embedding

In [3]:
# to communicate with google spreadsheet...
import gspread
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

s = sddk.cloudSession("sciencedata.dk")
# establish connection with gogglesheets...
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict") # or load it from a local storage: json.load(open("../../ServiceAccountsKey.json", "r"))
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
PIPA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1rV4t0_UV_wcx--UAHVwkqB8Wa_5n9mnpV05yGG1OHqk/edit?usp=sharing")

endpoint variable has been configured to: https://sciencedata.dk/files/


In [22]:
cgl = pd.read_json("../data/large_data/cgl.json")
cgl.head()

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmata,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*,conc_lype
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...",71863,None,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",31,0,6,25,"[[μέγας, κινδυνεύοντας, δέχομαι, ἀείμνηστος, μ..."
6,tlg0006.tlg001.perseus-grc2.xml,Euripides,Cyclops,4141,tlg0006,tlg0006.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...",2535,None,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...","[[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθεν...",7,0,0,1,"[[βοτόν, οὔτις, θύω, θεός, μέγας, γαστήρ, δαίμ..."
7,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,6272,tlg0006,tlg0006.tlg004,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...",3545,None,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...","[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...",11,0,1,2,"[[εἰμί, πολύς, χαίρω, δυσφημέω, ἅζομαι, θέα, σ..."
8,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,8257,tlg0006,tlg0006.tlg005,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...",4898,None,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...","[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...",8,3,8,8,"[[κοιτάζω, λέχος, σός, ναυβάτης, τὶς, πλέω, κρ..."
9,tlg0006.tlg006.perseus-grc2.xml,Euripides,Ἀνδρομάχη,7397,tlg0006,tlg0006.tlg006,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...",4420,None,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...","[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...",5,0,7,4,"[[Τροία, πράσσω, μηδείς, ὅδε, αὐχέω, πράσσω, δ..."


In [4]:
cgl_embeddings = pickle.load(open("../data/large_data/cgl_embeddings.pkl", "rb"))

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [14]:
len(range(1, len(cgl_embeddings[1])+1))

5000

In [42]:
word_index = dict(zip(cgl_embeddings[1],range(1, len(cgl_embeddings[1])+1)))
list(word_index.items())[:10]

[('εἰμί', 1),
 ('οὗτος', 2),
 ('λέγω', 3),
 ('ἔχω', 4),
 ('γίγνομαι', 5),
 ('πολύς', 6),
 ('ἄλλος', 7),
 ('αὐτός', 8),
 ('πᾶς', 9),
 ('ποιέω', 10)]

In [20]:
# we will generate the concordances again, now focusing on words in the embeddings only
def get_concordances(wordlist, keyword, window, vocabulary=None):
    half = int(window / 2)
    if vocabulary != None:
        wordlist = [w for w in wordlist if w in vocabulary]
    keyword_indices = [el[0] for el in enumerate(wordlist) if el[1]==keyword]
    concordances = [wordlist[i-half:i+half+1] for i in keyword_indices]
    concordances = [c for c in concordances if len(c)==window]
    return concordances

In [45]:
cgl["conc_lype"] = cgl["lemmata_repl"].apply(lambda x: get_concordances(x, "λύπ*", 31, vocabulary=cgl_embeddings[1]))

In [46]:
concs_series = cgl.apply(lambda row: [(row["author_id"], conc) for conc in row["conc_lype"]], axis=1)

In [47]:
concs_tups = [conc for conclist in concs_series for conc in conclist]
print(concs_tups[:3])

[('tlg0003', ['ἐπικουρία', 'ποιέω', 'μέγας', 'δέχομαι', 'μαρτύριον', 'χάρις', 'κατατίθημι', 'ναυτικός', 'κτέομαι', 'πολύς', 'σκέπτομαι', 'εὐπραξία', 'σπάνιος', 'τὶς', 'πολέμιος', 'λύπ*', 'πολύς', 'χρῆμα', 'χάρις', 'τιμάω', 'δύναμις', 'προσγίγνομαι', 'οὗτος', 'πάρειμι', 'κίνδυνος', 'δαπάνη', 'φέρω', 'πολύς', 'ἀρετή', 'ἐπαμύνω', 'χάρις']), ('tlg0003', ['τοιοῦτος', 'πόλις', 'Λακεδαιμόνιος', 'οἴομαι', 'ἡσυχία', 'ἄνθρωπος', 'πολύς', 'ἀρκέω', 'παρασκευή', 'δίκαιος', 'πράσσω', 'γνώμη', 'ἀδικέω', 'δῆλος', 'εἰμί', 'λύπ*', 'ἄλλος', 'αὐτός', 'ἀμύνω', 'βλάπτω', 'ἴσος', 'νέμω', 'πόλις', 'ὅμοιος', 'τυγχάνω', 'οὗτος', 'δηλόω', 'ἐπιτήδευμα', 'εἰμί', 'ἀνάγκη', 'τέχνη']), ('tlg0003', ['πᾶς', 'μέγας', 'κίνδυνος', 'τίθημι', 'Λακεδαιμόνιος', 'Πελοπόννησος', 'πόλις', 'ὠφέλιμος', 'καθίστημι', 'ἐξηγέομαι', 'ὑπομένω', 'πᾶς', 'ἡγεμονία', 'οἶδα', 'ἥσσων', 'λύπ*', 'γίγνομαι', 'συμμαχέω', 'ἄρχω', 'αὐτός', 'κινδυνεύω', 'θαυμαστός', 'οὐδείς', 'ποιέω', 'ἀνθρώπειος', 'τρόπος', 'ἀρχή', 'δέχομαι', 'οὗτος', 'ἀνίημι', 'μέ

In [50]:
concs_tups = [(conc[0],  np.array([word_index[w] for w in conc[1]])) for conc in concs_tups]
concs_tups[:3]

[('tlg0003',
  array([2877,   10,   18,  235, 2049,  193, 1264,  690,  421,    6,  452,
         4380, 2096,   22,  124,  208,    6,   88,  193,  306,   79, 1498,
            2,   92,  239, 1419,   63,    6,   97, 4892,  193])),
 ('tlg0003',
  array([  17,   13,  114,   45,  613,   24,    6, 1125,  870,   56,   57,
          189,  145,   89,    1,  208,    7,    8,  605,  488,  130,  572,
           13,  133,   74,    2,  319,  798,    1,   52,  153])),
 ('tlg0003',
  array([   9,   18,  239,   83,  114,  711,   13,  812,  142, 2251,  529,
            9, 1616,   31,  134,  208,    5,  681,   84,    8,  409,  629,
           15,   10, 2497,   68,   41,  235,    2, 1163,   18]))]

In [59]:
indices = np.arange(len(concs_tups))
np.random.seed(0)
np.random.shuffle(indices)

In [66]:
data = np.array([conc[1] for conc in concs_tups])
labels = []
for conc in concs_tups:
    if conc[0] == "tlg0086":
        labels.append(1)
    else:
        labels.append(0)
labels = np.array(labels)

In [67]:
data = data[indices]
labels = labels[indices]

In [87]:
training_samples = int(len(data) * 0.75)

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:]
y_val = labels[training_samples:]

In [88]:
embedding_matrix = cgl_embeddings[3]
embedding_matrix

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
εἰμί,40.921943,23.975110,6.083791,1.724597,0.787708,1.395476,10.085121,-3.512090,-2.466157,-0.479203,...,0.135525,0.116194,0.038779,0.028518,0.048852,0.063003,-0.006917,-0.285075,0.083432,0.050213
οὗτος,40.370998,23.444483,5.179671,1.611451,0.341264,1.735556,9.830402,-3.353345,-2.345492,-0.181414,...,0.139695,0.078060,0.127660,0.069392,0.060385,0.249957,0.070054,-0.194425,-0.004014,0.040241
λέγω,39.001855,22.120811,3.711950,0.918392,1.452316,0.077037,9.391213,-2.759150,-2.398183,0.214704,...,-0.031307,0.068023,0.000057,0.069156,0.111072,0.038494,0.002585,-0.019338,0.269553,-0.130885
ἔχω,39.804302,23.024422,6.199601,2.275942,0.625015,1.243543,9.147309,-3.485462,-2.581793,-0.275819,...,0.024101,0.063533,0.134114,0.179058,0.043486,0.113336,-0.036198,-0.342098,0.209940,-0.073624
γίγνομαι,39.347879,22.489298,5.427730,1.608740,-0.299176,1.453300,8.521668,-3.282159,-2.755167,-0.361946,...,0.017706,0.150147,-0.100498,0.024932,-0.241511,0.180704,-0.044182,-0.225451,-0.009623,0.082624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
κύαθος,2.306894,-1.252014,1.967848,0.639823,-0.830738,1.400087,0.623265,-0.022984,1.364765,-1.015133,...,0.029445,0.035054,-0.064084,-0.022086,-0.026919,-0.029758,-0.059165,0.081717,0.070341,0.065586
σκυτεύς,2.426349,-1.921372,0.401636,-0.442950,-0.050278,0.079601,1.183014,-0.539222,0.169868,0.727299,...,-0.379041,0.069706,0.277146,0.071542,-0.010878,0.112946,-0.114023,0.315502,-0.062303,-0.036280
παραμίγνυμι,2.460930,-1.073599,2.207248,0.563545,-0.814028,1.938490,0.646300,-0.572682,2.623897,-1.114314,...,-0.005971,-0.170639,-0.140238,0.136425,0.311922,0.270307,-0.047148,-0.198601,-0.085457,0.311602
ὀφθαλμία,1.968229,-1.475017,1.460325,-0.047004,-0.329035,0.401820,0.145133,-0.578576,-0.902634,0.095910,...,-0.229633,-0.162759,-0.336879,-0.035314,0.135483,-0.145982,0.128570,-0.032265,-0.125454,-0.178760


In [89]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(5000,  150, input_length=31))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 31, 150)           750000    
                                                                 
 flatten_3 (Flatten)         (None, 4650)              0         
                                                                 
 dense_6 (Dense)             (None, 32)                148832    
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 898,865
Trainable params: 898,865
Non-trainable params: 0
_________________________________________________________________


In [90]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [91]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
27/27 [==============================] - 1s 5ms/step - loss: 1.1336 - acc: 0.5684 - val_loss: 0.6552 - val_acc: 0.6000
Epoch 2/10
27/27 [==============================] - 0s 2ms/step - loss: 0.4008 - acc: 0.8094 - val_loss: 0.5959 - val_acc: 0.6737
Epoch 3/10
27/27 [==============================] - 0s 2ms/step - loss: 0.2102 - acc: 0.9205 - val_loss: 0.5631 - val_acc: 0.7053
Epoch 4/10
27/27 [==============================] - 0s 2ms/step - loss: 0.0841 - acc: 0.9848 - val_loss: 1.2212 - val_acc: 0.5228
Epoch 5/10
27/27 [==============================] - 0s 2ms/step - loss: 0.0632 - acc: 0.9778 - val_loss: 2.1046 - val_acc: 0.6421
Epoch 6/10
27/27 [==============================] - 0s 1ms/step - loss: 0.0403 - acc: 0.9895 - val_loss: 0.7022 - val_acc: 0.7053
Epoch 7/10
27/27 [==============================] - 0s 2ms/step - loss: 0.0123 - acc: 1.0000 - val_loss: 1.7742 - val_acc: 0.6596
Epoch 8/10
27/27 [==============================] - 0s 1ms/step - loss: 0.0403 - acc: 0.98